In [ ]:
from langchain_openai.chat_models import AzureChatOpenAI  
from langchain_openai import AzureOpenAIEmbeddings
from langchain_weaviate.vectorstores import WeaviateVectorStore

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from vectorvault.chat.pages.utils import PROMPT

import weaviate
import os
import json

COLLECTION_NAME = "recipes"

In [ ]:
question = "Whats the recipe for chocolate cookies?"

openai_client = AzureChatOpenAI(
        model_name="gpt-4", 
        deployment_name = "gpt-4",
        api_version="2024-02-01",
    )

with  weaviate.connect_to_local(
    port=8083,
    grpc_port=50051,
    headers = {"X-Azure-Api-Key":  os.getenv('OPENAI_API_KEY')},
) as weaviate_client:
    
    weaviate_db = WeaviateVectorStore(
        client=weaviate_client, 
        index_name=COLLECTION_NAME, 
        text_key="chunk", 
        embedding=AzureOpenAIEmbeddings(model="text-embedding-3-large")
    )
    
    combine_docs_chain = create_stuff_documents_chain(openai_client, PROMPT)
    
    rag_chain = create_retrieval_chain(weaviate_db.as_retriever(), combine_docs_chain)

    output = rag_chain.invoke({
        "input": question,
        "chat_history": None
        }
    )

response = json.loads(output["answer"])
print(response)